In [1]:
from pdb import run
from unittest import case
import emcee
import corner
import pickle
from tqdm import tqdm
import numpy as np
from qutip import *
from multiprocess import Pool
import matplotlib.pyplot as plt
from scipy.optimize import minimize, differential_evolution, basinhopping
from IPython.display import display, Math
from scipy.stats import chi2
from protocolsClass import Hamiltonian_Fitter,State


In [2]:
ground_meas = np.array([7512786.3, 6847127.7, 6180056.7, 5510661.7, 4837199., 4156107.7, 3459061.2, 2730533.1, 1787541.6]) * 1e-3 # [kHz]
manu_ramsey_meas = np.array([-134296, -133678, -132898, -131896, -130532, -128697, -125952, -124533, -88889]) * 1e-3 # [kHz]
full_meas = np.concatenate((ground_meas,manu_ramsey_meas))

d_ground_meas = np.array([1,1,1,1,1,1,1,1,1]) * 1e-3 # [kHz]
d_manu_ramsey_meas = np.array([1,1,1,1,1,1,1,1,1]) * 20e-3 # [kHz]
d_full_meas = np.concatenate((d_ground_meas,d_manu_ramsey_meas))

A_perp_meas = 105 # [kHz] | Measured through the Raman Rabi experiment
A_simu = np.array([[-436.6,    0.,   -41.3],
                    [  -0.,  -448.4,    0. ],
                    [ -88.5,    0.,   129.8]])


fitter_ground = Hamiltonian_Fitter(ground_meas,d_ground_meas,State.Ground)
fitter_excited = Hamiltonian_Fitter(ground_meas + manu_ramsey_meas,d_ground_meas,State.Excited)
fitter_full = Hamiltonian_Fitter(full_meas,d_full_meas,State.Full, meas_Aperp = A_perp_meas,simu_A= A_simu)

In [3]:
guess_exicted = [446.3, -237.1,  149.5,   10.65, 0.6]
guess_exicted = [ 4.43614051e+02 ,-2.36794019e+02, 1.49286163e+02, 1.59948409e+01,1.12819327e-01] #1RUN 
guess_exicted = [ 4.43614051e+02 ,-2.36794019e+02, 1.49286163e+02, 1.59948409e+01,1.12819327e-01] #1RUN 
guess_exicted = [ 4.43613977e+02, -2.36793936e+02, 1.49286932e+02, 1.59926691e+01, 1.11464109e-01]

guess_ground = [4.60832e+02, -2.37368e+02,  1.25900e+00, -4.00000e-03, 1.53400e+00] 
guess_ground = [ 456.14443873 ,-237.12801178 , 148.54641495 ,   7.46758762 , 374.08954251] # RUN1
guess_ground  =  [ 456.0840477 , -237.22287091 , 149.38269055  ,  8.26172301 , 374.01006077] #RUN2
guess_ground = [ 456.08283005, -237.22462143,  149.39880504,    8.29825619,  374.0529795 ] #RUN3


guess_full = np.array([4.498e+02,  1.299e+02, -2.370e+02, -1.190e+01, -1.492e+02,  1.500e+00, -8.000e-01,  1.000e-01])

In [4]:
h = fitter_full.Full_hamiltonian(guess_full)
e,g = fitter_full.get_transitions_separated(h.eigenenergies())

fitter_full.Full_hamiltonian(guess_full)
fitter_full.get_log_likelihood_separated(guess_full)

-890873.1137048828

In [5]:
# Lamb shift
rel_electron_freq = np.cumsum([0, 10]) # [kHz]
g=5.24            # [kHz]
kappa=700         # [kHz]
lamb_shift_meas = rel_electron_freq * g**2 / (kappa**2/4 + rel_electron_freq**2) # [kHz]

rel_electron_freq

array([ 0, 10])

In [10]:
%pdb on
fitter_full.Full_hamiltonian(guess_full)
fitter_full.run_MCMC(guess_full)

Automatic pdb calling has been turned ON


TypeError: cannot unpack non-iterable method object

> c:\users\leon\onedrive\documents\spectroscopy_hamiltonian_fitter-\protocolsclass.py(170)Full_hamiltonian()
    168     # Define the Hamiltonian
    169     def Full_hamiltonian(self,x: np.ndarray) -> Qobj:
--> 170         Bz, A, D, S1, S2, delta, alpha, Dz = x
    171         return self.zeeman_full_hamiltonian(Bz) +\
    172             self.hyperfine_hamiltonian(A) +\

*** NameError: name 'fitter_full' is not defined
*** NameError: name 'fitter_full' is not defined
*** NameError: name 'fitter_full' is not defined
